In [1]:
from fuzzywuzzy import process
import pandas as pd
import re

In [3]:
df_dis = pd.read_csv("diseasedata/disease_list.csv",index_col=0, header=0)
diseases = df_dis['name'].str.strip().str.replace(' +',' ').str.lower()
urls = df_dis['ref'].apply(lambda x: "http://www.mayoclinic.org/diseases-conditions"+x)
d_list = diseases.loc[:373].tolist()
u_list = urls.loc[:373].tolist()

df_dis = pd.read_csv("diseasedata/disease_list_cdc.csv",index_col=None, header=0)
diseases = df_dis['name'].str.strip().str.replace(' +',' ').str.lower()
urls = df_dis['ref']
d_list = d_list+diseases.tolist()
u_list = u_list+urls.tolist()

uniquer=[]
lookup={}
disease_list=[]
to_syn = [(re.sub('see also ','',a),b) for (a,b) in zip(d_list,u_list) if \
          (('vaccination' not in a) and \
           ('vaccines' not in a) and \
           ('-' not in a) and \
           ('zika virus infection and pregnancy' not in a) and \
           ('cdc vision and mission' not in a) and \
           ('health disparities' not in a) and \
           ('health history' not in a) and \
           ('biopsy' not in a) and \
           ('ra' != a))]



for i,c in enumerate(to_syn):
    if(i%1000==0):
        print i
    if(c[0] not in uniquer):
        uniquer.append(c[0])
        match = process.extractOne(c[0], lookup.keys())
        if(match and match[1]>=88):
            spot=lookup[match[0]]
            disease_list[spot][2].append(c[0])
            lookup[c[0]]=spot
        else:
            lookup[c[0]]=len(disease_list)
            disease_list.append(c+([c[0]],))



0
1000


In [ ]:
#sorted([(c[0],len(c[2]),c[2]) for c in disease_list],key=lambda x: x[1],reverse=True)[:25]

In [4]:
import cloudpickle
with open("pickles/disease_list.pkl", 'wb') as handle:
    cloudpickle.dump(disease_list, handle)
with open("pickles/disease_lookup.pkl", 'wb') as handle:
    cloudpickle.dump(lookup, handle)